In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [5]:
train = pd.read_csv('lr_train.csv', sep=";")
train.head() #0_x is ev charging current, 0_y is baseload current

,Unnamed: 0,Unnamed: 0.1,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,0_current,1_current,2_current,3_current,4_current,5_current,6_current,7_current,8_current,9_current,10_current,11_current,12_current,13_current,14_current,15_current,16_current,17_current,18_current,19_current,20_current,21_current,22_current,23_current,total_chargers
0,0,24,138,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",79301:001:0342,58.383253,26.764144,6474748.69,661646.72,100,0.0,1.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,43.085689,27.219903,37.226985,33.587168,...,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,42.328719,35.826586,48.375035,42.896676,50.285689,34.419903,37.226985,33.587168,37.732611,38.310797,50.487424,47.217521,43.578974,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,49.528719,43.026586,48.375035,42.896676,1.0
1,1,16,15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",79502:019:0018,58.370127,26.689523,6473103.15,657342.63,60,2.0,5.0,84.2,40.2,33.0,22.0,0.0,38.5,0.0,0.0,0.0,45.7,66.0,0.0,44.0,0.0,22.0,0.0,22.0,23.7,7.2,67.7,0.0,55.0,23.7,7.2,79.995283,68.110978,48.466686,70.917465,...,70.624213,72.380735,71.295732,74.266680,62.560722,63.129722,71.452590,80.789491,78.605374,83.275353,69.291341,87.294980,90.960876,68.542676,73.093613,164.195283,108.310978,81.466686,92.917465,43.575427,79.514364,75.954881,53.692525,93.099298,116.324213,138.380735,71.295732,118.266680,62.560722,85.129722,71.452590,102.789491,102.305374,90.475353,136.991341,87.294980,145.960876,92.242676,80.293613,7.0
2,2,8,193,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",79512:032:0012,58.390462,26.727509,6475460.14,659468.49,160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,0.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,36.859454,24.384204,42.659863,34.714995,...,40.462273,45.447998,51.700038,54.268805,42.412902,57.000000,37.749564,46.723661,48.290400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,36.859454,24.384204,42.659863,34.714995,29.641593,27.318412,41.759503,42.054959,51.676428,47.662273,45.447998,51.700038,54.268805,49.612902,64.200000,37.749564,46.723661,55.490400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,1.0
3,3,15,168,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",79514:014:0021,58.385042,26.730641,6474849.44,659676.18,160,1.0,4.0,51.7,29.7,23.1,6.6,6.6,0.0,0.0,7.2,0.0,6.6,23.7,6.6,16.5,23.1,13.8,6.6,23.7,6.6,0.0,0.0,28.6,51.7,0.0,0.0,25.561742,34.515704,42.755236,34.533194,...,50.798240,39.649469,39.679657,43.217380,43.789523,42.744132,55.933350,44.871371,48.399381,52.284646,43.269583,44.919531,37.376938,36.109098,45.307183,77.261742,64.215704,65.855236,41.133194,40.794738,32.276497,52.200273,64.200000,57.000000,57.398240,63.349469,46.279657,59.717380,66.889523,56.544132,62.533350,68.571371,54.999381,52.284646,43.269583,73.519531,89.076938,36.109098,45.307183,5.0
4,4,12,59,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",79507:022:0008,58.378215,26.714904,6474067.16,658809.66,160,3.0,2.0,13.8,7.2,0.0,6.6,6.6,13.8,0.0,0.0,44.0,50.6,22.0,28.6,50.6,23.1,60.5,28.6,23.1,0.0,0.0,0.0,13.8,0.0,6.6,0.0,152.551035,148.046867,96.830484,146.926278,...,173.291836,142.489328,168.850415,159.013063,159.637415,164.057110,122.495332,178.333348,147.735354,163.163628,167.412285,165.004095,161.251106,172.177444,155.879436,166.351035,155.246867,96.830484,153.526278,143.813011,100.654798,139.900387,139.283876,184.451160,223.891836,164.489328,197.450415,209.613063,182.737415,224.557110,151.095332,201.433348,147.735354,163.163628,167.412285,178.804095,161.251106,178.777444,155.879436,5.0


In [6]:
test = pd.read_csv('lr_test.csv', sep=";")
test.head() #0_x is ev charging current, 0_y is baseload current

,Unnamed: 0,Unnamed: 0.1,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,0_current,1_current,2_current,3_current,4_current,5_current,6_current,7_current,8_current,9_current,10_current,11_current,12_current,13_current,14_current,15_current,16_current,17_current,18_current,19_current,20_current,21_current,22_current,23_current,total_chargers
0,0,17,106,"15, Kopli, Variku, Tartu linn, Tartu, Tartu li...",79509:025:0008,58.348811,26.708916,6470771.33,658583.32,100,1.0,1.0,16.5,16.5,16.5,0.0,0.0,0.0,0.0,0.0,0.0,23.7,7.2,0.0,0.0,0.0,0.0,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,84.363099,87.203655,81.551798,79.853646,...,152.000000,114.895037,140.790479,133.615248,116.041691,121.863803,112.625463,110.742833,126.418970,98.221259,132.273297,103.484166,101.497234,99.091078,114.843304,100.863099,103.703655,98.051798,79.853646,85.161289,101.421970,120.300720,117.936946,119.215644,175.700000,122.095037,140.790479,133.615248,116.041691,121.863803,112.625463,110.742833,133.618970,98.221259,132.273297,103.484166,101.497234,99.091078,114.843304,2.0
1,1,13,7,"15, Ränikivi, Tammelinn, Tartu linn, Tartu, Ta...",79504:044:0020,58.362871,26.711394,6472375.36,658641.40,60,1.0,7.0,94.2,28.2,6.6,7.2,0.0,0.0,0.0,16.5,0.0,38.5,7.2,22.0,0.0,7.2,7.2,38.5,0.0,7.2,22.0,7.2,29.2,22.0,29.2,28.6,54.697513,35.803199,69.880869,68.990245,...,77.869466,86.600795,74.121484,48.742505,89.572258,83.748550,84.752867,87.988713,85.651310,85.378897,84.402053,89.357965,69.109202,59.464003,63.653381,148.897513,64.003199,76.480869,76.190245,81.613506,47.080922,86.371732,90.180082,77.014148,116.369466,93.800795,96.121484,48.742505,96.772258,90.948550,123.252867,87.988713,92.851310,107.378897,91.602053,118.557965,91.109202,88.664003,92.253381,8.0
2,2,4,30,"64d, Tamme pst, Tammelinn, Tartu linn, Tartu, ...",79504:058:0032,58.359741,26.698696,6471979.51,657934.06,160,3.0,5.0,54.0,54.0,16.5,23.1,6.6,13.8,0.0,13.8,0.0,30.3,14.4,7.2,20.4,0.0,13.8,7.2,13.8,6.6,13.8,23.1,6.6,47.4,7.2,30.3,84.439416,122.545329,90.470726,120.791015,...,166.854489,132.949124,164.448438,145.199517,190.000000,109.285808,146.370440,160.213228,159.446888,141.163970,164.290339,190.000000,174.801916,149.993054,154.378589,138.439416,176.545329,106.970726,143.891015,87.030400,115.769547,141.581941,163.549522,189.286543,197.154489,147.349124,171.648438,165.599517,190.000000,123.085808,153.570440,174.013228,166.046888,154.963970,187.390339,196.600000,222.201916,157.193054,184.678589,8.0
3,3,29,194,"2a, Lubja, Ülejõe, Tartu linn, Tartu, Tartu li...",79301:001:0749,58.390196,26.716788,6475398.26,658858.08,100,2.0,3.0,33.0,55.0,33.0,0.0,0.0,0.0,0.0,7.2,22.0,29.2,7.2,29.2,29.2,30.9,29.2,7.2,22.0,14.4,16.5,38.5,33.0,16.5,16.5,0.0,95.618853,121.410330,124.239128,172.485954,...,140.235295,139.341308,135.442568,109.607580,124.901294,156.648397,170.432186,148.708193,107.179530,171.347742,137.535151,139.055106,162.863514,170.526774,136.501670,128.618853,176.410330,157.239128,172.485954,128.902587,152.225443,165.380419,187.967037,155.765616,169.435295,146.541308,164.642568,138.807580,155.801294,185.848397,177.632186,170.708193,121.579530,187.847742,176.035151,172.055106,179.363514,187.026774,136.501670,5.0
4,4,35,103,"59B, Jalaka, Ropka tööstusrajoon, Tartu linn, ...",79510:033:0012,58.346660,26.722654,6470566.42,659376.72,100,1.0,1.0,22.0,0.0,0.0,0.0,0.0,0.0,13.8,0.0,6.6,13.8,7.2,0.0,0.0,7.2,7.2,0.0,13.8,0.0,0.0,0.0,22.0,0.0,0.0,0.0,117.605310,92.244109,98.276301,103.096433,...,102.450228,108.722290,106.804187,115.620198,91.374732,134.976471,96.880598,121.850234,127.985356,117.952242,84.971010,103.488222,102.493993,79.221729,106.508075,139.605310,92.244109,98.276301,103.096433,73.742917,91.210275,148.828626,113.165050,141.898434,116.250228,115.922290,106.804187,115.620198,98.574732,142.17

In [4]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

for i in range(24):
  train[str(i)+"_current"] = train[str(i)+"_x"]+train[str(i)+"_y"]
  test[str(i)+"_current"] = test[str(i)+"_x"]+test[str(i)+"_y"]
train["total_chargers"] = train["public_chargers"] + train["home_chargers"]
test["total_chargers"] = test["public_chargers"] + test["home_chargers"]
train.head()

,Unnamed: 0,Unnamed: 0.1,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,0_current,1_current,2_current,3_current,4_current,5_current,6_current,7_current,8_current,9_current,10_current,11_current,12_current,13_current,14_current,15_current,16_current,17_current,18_current,19_current,20_current,21_current,22_current,23_current,total_chargers
0,0,24,138,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",79301:001:0342,58.383253,26.764144,6474748.69,661646.72,100,0.0,1.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,43.085689,27.219903,37.226985,33.587168,...,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,42.328719,35.826586,48.375035,42.896676,50.285689,34.419903,37.226985,33.587168,37.732611,38.310797,50.487424,47.217521,43.578974,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,49.528719,43.026586,48.375035,42.896676,1.0
1,1,16,15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",79502:019:0018,58.370127,26.689523,6473103.15,657342.63,60,2.0,5.0,84.2,40.2,33.0,22.0,0.0,38.5,0.0,0.0,0.0,45.7,66.0,0.0,44.0,0.0,22.0,0.0,22.0,23.7,7.2,67.7,0.0,55.0,23.7,7.2,79.995283,68.110978,48.466686,70.917465,...,70.624213,72.380735,71.295732,74.266680,62.560722,63.129722,71.452590,80.789491,78.605374,83.275353,69.291341,87.294980,90.960876,68.542676,73.093613,164.195283,108.310978,81.466686,92.917465,43.575427,79.514364,75.954881,53.692525,93.099298,116.324213,138.380735,71.295732,118.266680,62.560722,85.129722,71.452590,102.789491,102.305374,90.475353,136.991341,87.294980,145.960876,92.242676,80.293613,7.0
2,2,8,193,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",79512:032:0012,58.390462,26.727509,6475460.14,659468.49,160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,0.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,36.859454,24.384204,42.659863,34.714995,...,40.462273,45.447998,51.700038,54.268805,42.412902,57.000000,37.749564,46.723661,48.290400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,36.859454,24.384204,42.659863,34.714995,29.641593,27.318412,41.759503,42.054959,51.676428,47.662273,45.447998,51.700038,54.268805,49.612902,64.200000,37.749564,46.723661,55.490400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,1.0
3,3,15,168,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",79514:014:0021,58.385042,26.730641,6474849.44,659676.18,160,1.0,4.0,51.7,29.7,23.1,6.6,6.6,0.0,0.0,7.2,0.0,6.6,23.7,6.6,16.5,23.1,13.8,6.6,23.7,6.6,0.0,0.0,28.6,51.7,0.0,0.0,25.561742,34.515704,42.755236,34.533194,...,50.798240,39.649469,39.679657,43.217380,43.789523,42.744132,55.933350,44.871371,48.399381,52.284646,43.269583,44.919531,37.376938,36.109098,45.307183,77.261742,64.215704,65.855236,41.133194,40.794738,32.276497,52.200273,64.200000,57.000000,57.398240,63.349469,46.279657,59.717380,66.889523,56.544132,62.533350,68.571371,54.999381,52.284646,43.269583,73.519531,89.076938,36.109098,45.307183,5.0
4,4,12,59,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",79507:022:0008,58.378215,26.714904,6474067.16,658809.66,160,3.0,2.0,13.8,7.2,0.0,6.6,6.6,13.8,0.0,0.0,44.0,50.6,22.0,28.6,50.6,23.1,60.5,28.6,23.1,0.0,0.0,0.0,13.8,0.0,6.6,0.0,152.551035,148.046867,96.830484,146.926278,...,173.291836,142.489328,168.850415,159.013063,159.637415,164.057110,122.495332,178.333348,147.735354,163.163628,167.412285,165.004095,161.251106,172.177444,155.879436,166.351035,155.246867,96.830484,153.526278,143.813011,100.654798,139.900387,139.283876,184.451160,223.891836,164.489328,197.450415,209.613063,182.737415,224.557110,151.095332,201.433348,147.735354,163.163628,167.412285,178.804095,161.251106,178.777444,155.879436,5.0


In [ ]:
print(np.unique(train["max_current"]),np.unique(test["max_current"]))

[ 60 100 160 200] [ 60 100 160]


In [8]:
# Per hour model 1
models = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_x']])
  pred = lr.predict(test[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]])
  models.append([lr, lr.score(test[["max_current", "public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_x']]), mean_squared_error(test[[str(i)+'_x']], pred, squared=False)])
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",model[1],"| RMSE",model[2])

print("Day average | R2", np.array(models)[:,1].mean(),"| RMSE", np.array(models)[:,2].mean())

Day average | R2 -0.4517463082193587 | RMSE 10.925397972186156


In [ ]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_x"])):
    print(i, el[0][0], el[1])

In [10]:
# Per hour model 2
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_x']])
  pred = lr.predict(test[["public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 -0.26289974416228595 | RMSE 10.423725187528833


In [12]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_x"])):
    print(i, el[0][0], el[1])

0 13.797653826012864 16.5
0 79.06353684495502 94.2
0 57.74274224355347 54.0
0 35.98878769376756 33.0
0 14.445422175219546 22.0
0 46.16612852538209 72.6
0 25.85168424546574 23.1
0 2.851496057078842 0.0
1 4.4733889865579 16.5
1 51.12740663257553 28.2
1 36.002070174012474 54.0
1 19.284700229069283 55.0
1 4.182886599581592 0.0
1 25.958267931556104 6.6
1 7.561441020350283 23.1
1 -3.685552561783134 0.0
2 4.799409347367891 16.5
2 34.58868527704338 6.6
2 23.910264676436974 16.5
2 11.39776500885672 33.0
2 3.5057049480780575 0.0
2 19.960488278914774 6.6
2 6.8158139987405875 6.6
2 -2.9879135183614274 0.0
3 4.076026353325911 0.0
3 17.485365888300187 7.2
3 15.955361651918158 23.1
3 13.418825211794234 0.0
3 5.172122264489747 0.0
3 10.240697100043944 0.0
3 7.414889565763042 0.0
3 2.9190743396716963 0.0
4 1.567460298019597 0.0
4 13.594086174547797 0.0
4 11.973050734620589 6.6
4 7.375083281756843 0.0
4 1.4176742250298533 0.0
4 6.097970216363019 0.0
4 2.7736978270714916 16.5
4 -0.3765779007946879 0.0
5 

In [178]:
# Per hour model 3
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  # Labels are the values we want to predict
  labels = np.array(train[str(i)+'_y'])
  # Remove the labels from the features
  # axis 1 refers to the columns
  features = train[["public_chargers", "home_chargers", str(i)+"_y", "max_current"]]
  # Saving feature names for later use
  feature_list = list(features.columns)
  # Convert to numpy array
  features = np.array(features)

  m = RandomForestRegressor()
  m.fit(features, labels)
  pred = m.predict(test[["public_chargers", "home_chargers", str(i)+"_y", "max_current"]])
  models.append(m)
  r2.append(m.score(test[["public_chargers", "home_chargers", str(i)+"_y", "max_current"]], test[[str(i)+'_y']]))
  rmse.append(mean_squared_error(test[[str(i)+'_y']], pred, squared=False))
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 0.9334367843043615 | RMSE 7.542217461877776


In [180]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_x"])):
    print(i, el[0], el[1])

0 76.7264590732342 16.5
0 59.59607045953767 94.2
0 81.21159346923008 54.0
0 104.45410353702847 33.0
0 108.01302782636593 22.0
0 58.31837790593783 72.6
0 134.17880679989622 23.1
0 77.99355428744072 0.0
1 89.17549604833818 16.5
1 34.50322749384948 28.2
1 128.3480024256697 54.0
1 122.60132472245185 55.0
1 91.03573611477091 0.0
1 58.259178649347895 6.6
1 123.75605959088202 23.1
1 103.61640308322961 0.0
2 82.29672262741255 16.5
2 71.38936798575266 6.6
2 89.5173984777451 16.5
2 116.88494572738344 33.0
2 95.02089858567373 0.0
2 73.17610443127364 6.6
2 110.88630284384928 6.6
2 113.50676327540519 0.0
3 74.34542123929062 0.0
3 67.91099243267307 7.2
3 116.04164484487401 23.1
3 138.08288266382894 0.0
3 104.7658030578804 0.0
3 68.81233571028265 0.0
3 108.74908292123166 0.0
3 104.60730305990535 0.0
4 83.38280096212412 0.0
4 83.54284153316571 0.0
4 85.7635747829853 6.6
4 124.84877820394249 0.0
4 72.06054766517713 0.0
4 64.58479512190743 0.0
4 115.53968911694214 16.5
4 89.91567860866373 0.0
5 100.2825

In [16]:
# Per hour model 4
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_x']])
  pred = lr.predict(test[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["latitude", "longitude", "public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 -0.3091843538918535 | RMSE 10.606164671025338


In [ ]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_x"])):
    print(i, el[0][0], el[1])

In [17]:
# Per hour model 5
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = LinearRegression()
  lr.fit(train[["public_chargers", "home_chargers"]], train[[str(i)+'_x']])
  pred = [max(0,pr[0]) for pr in lr.predict(test[["public_chargers", "home_chargers"]])]
  models.append(lr)
  r2.append(lr.score(test[["public_chargers", "home_chargers"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 -0.26632181986555786 | RMSE 9.865394332564243


In [ ]:
print("ev charge prediction, actual ev charge")
for i in range(24):
  for el in zip(predictions[i],np.array(test[str(i)+"_x"])):
    print(i, el[0], el[1])

In [25]:
# Per hour model 6
from sklearn.linear_model import ElasticNet

models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = ElasticNet()
  lr.fit(train[["latitude", "longitude", "max_current", "public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_x']])
  pred = lr.predict(test[["latitude", "longitude", "max_current", "public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["latitude", "longitude", "max_current", "public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

Day average | R2 -0.3005340005889618 | RMSE 10.398412247810915


In [27]:
for i, model in enumerate(models):
  #print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])
  print("Model",model.coef_,model.intercept_)

Model [ 0.          0.         -0.0892128  -0.          9.72027113  0.07524548] [11.83275984]
Model [ 0.         -0.         -0.03823939  1.43350322  6.57905634 -0.01739275] [5.08429275]
Model [ 0.          0.         -0.03080476  0.03112107  4.27557638 -0.04579849] [9.65647793]
Model [-0.          0.         -0.03690159  0.          2.0381932   0.06698852] [2.25154212]
Model [-0.          0.         -0.03316446  0.63314648  1.81376056  0.03320473] [1.43010581]
Model [ 0.         -0.         -0.06334233  0.3139506   1.8086634   0.02334258] [6.87119099]
Model [ 0.         -0.          0.00994522  1.23509539 -0.21763369 -0.00616456] [-0.44458363]
Model [ 0.         -0.         -0.01282373  3.62913993 -0.55646799 -0.00380139] [4.9627883]
Model [ 0.         -0.          0.04101274  3.86829106 -1.30420858  0.0680053 ] [-6.44766095]
Model [ 0.00000000e+00 -0.00000000e+00  2.65035915e-02  1.36054374e+01
 -2.86828871e+00 -1.24092893e-02] [9.74048172]
Model [ 0.         -0.         -0.03048644 

In [ ]:
for i in range(24):
  for el in zip(predictions[i], np.array(test[str(i)+"_x"])):
    print(i, el)

In [19]:
# Per hour model 7
from sklearn.linear_model import Ridge

models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  lr = Ridge()
  lr.fit(train[["public_chargers", "home_chargers", str(i)+"_y"]], train[[str(i)+'_x']])
  pred = lr.predict(test[["public_chargers", "home_chargers", str(i)+"_y"]])
  models.append(lr)
  r2.append(lr.score(test[["public_chargers", "home_chargers", str(i)+"_y"]], test[[str(i)+'_x']]))
  rmse.append(mean_squared_error(test[[str(i)+'_x']], pred, squared=False))
  predictions.append(pred)

print("Day average | R2", np.mean(r2),"| RMSE", np.mean(rmse))

# for i, model in enumerate(models):
#   print("Hour",i+1,"| R2",r2[i],"| RMSE", rmse[i])

Day average | R2 -0.24893728752808553 | RMSE 10.385799403526038


## Best model

The best model turned out to be model 3.

## Restricted charging

In [29]:
# Per hour model 3
models = []
r2 = []
rmse = []
predictions = []

for i in range(24):
  # Labels are the values we want to predict
  labels = np.array(train[str(i)+'_y'])
  # Remove the labels from the features
  features = train[["public_chargers", "home_chargers", str(i)+"_y", "max_current"]]
  # Saving feature names for later use
  feature_list = list(features.columns)
  # Convert to numpy array
  features = np.array(features)

  m = RandomForestRegressor()
  m.fit(features, labels)
  models.append(m)

In [87]:
all_substations = pd.concat([train, test])
all_substations.head()

,Unnamed: 0,Unnamed: 0.1,index,address,cadaster,latitude,longitude,x,y,max_current,public_chargers,home_chargers,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,0_y,1_y,2_y,3_y,...,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,0_current,1_current,2_current,3_current,4_current,5_current,6_current,7_current,8_current,9_current,10_current,11_current,12_current,13_current,14_current,15_current,16_current,17_current,18_current,19_current,20_current,21_current,22_current,23_current,total_chargers
0,0,24,138,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",79301:001:0342,58.383253,26.764144,6474748.69,661646.72,100,0.0,1.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,43.085689,27.219903,37.226985,33.587168,...,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,42.328719,35.826586,48.375035,42.896676,50.285689,34.419903,37.226985,33.587168,37.732611,38.310797,50.487424,47.217521,43.578974,45.743862,36.197258,45.500228,41.822294,48.759855,39.381768,41.946460,48.407753,48.198700,46.375032,48.143367,49.528719,43.026586,48.375035,42.896676,1.0
1,1,16,15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",79502:019:0018,58.370127,26.689523,6473103.15,657342.63,60,2.0,5.0,84.2,40.2,33.0,22.0,0.0,38.5,0.0,0.0,0.0,45.7,66.0,0.0,44.0,0.0,22.0,0.0,22.0,23.7,7.2,67.7,0.0,55.0,23.7,7.2,79.995283,68.110978,48.466686,70.917465,...,70.624213,72.380735,71.295732,74.266680,62.560722,63.129722,71.452590,80.789491,78.605374,83.275353,69.291341,87.294980,90.960876,68.542676,73.093613,164.195283,108.310978,81.466686,92.917465,43.575427,79.514364,75.954881,53.692525,93.099298,116.324213,138.380735,71.295732,118.266680,62.560722,85.129722,71.452590,102.789491,102.305374,90.475353,136.991341,87.294980,145.960876,92.242676,80.293613,7.0
2,2,8,193,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",79512:032:0012,58.390462,26.727509,6475460.14,659468.49,160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.2,7.2,0.0,0.0,0.0,7.2,7.2,0.0,0.0,7.2,0.0,0.0,0.0,0.0,0.0,0.0,36.859454,24.384204,42.659863,34.714995,...,40.462273,45.447998,51.700038,54.268805,42.412902,57.000000,37.749564,46.723661,48.290400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,36.859454,24.384204,42.659863,34.714995,29.641593,27.318412,41.759503,42.054959,51.676428,47.662273,45.447998,51.700038,54.268805,49.612902,64.200000,37.749564,46.723661,55.490400,54.797779,52.671071,47.350626,45.199859,47.173830,41.957258,1.0
3,3,15,168,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",79514:014:0021,58.385042,26.730641,6474849.44,659676.18,160,1.0,4.0,51.7,29.7,23.1,6.6,6.6,0.0,0.0,7.2,0.0,6.6,23.7,6.6,16.5,23.1,13.8,6.6,23.7,6.6,0.0,0.0,28.6,51.7,0.0,0.0,25.561742,34.515704,42.755236,34.533194,...,50.798240,39.649469,39.679657,43.217380,43.789523,42.744132,55.933350,44.871371,48.399381,52.284646,43.269583,44.919531,37.376938,36.109098,45.307183,77.261742,64.215704,65.855236,41.133194,40.794738,32.276497,52.200273,64.200000,57.000000,57.398240,63.349469,46.279657,59.717380,66.889523,56.544132,62.533350,68.571371,54.999381,52.284646,43.269583,73.519531,89.076938,36.109098,45.307183,5.0
4,4,12,59,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",79507:022:0008,58.378215,26.714904,6474067.16,658809.66,160,3.0,2.0,13.8,7.2,0.0,6.6,6.6,13.8,0.0,0.0,44.0,50.6,22.0,28.6,50.6,23.1,60.5,28.6,23.1,0.0,0.0,0.0,13.8,0.0,6.6,0.0,152.551035,148.046867,96.830484,146.926278,...,173.291836,142.489328,168.850415,159.013063,159.637415,164.057110,122.495332,178.333348,147.735354,163.163628,167.412285,165.004095,161.251106,172.177444,155.879436,166.351035,155.246867,96.830484,153.526278,143.813011,100.654798,139.900387,139.283876,184.451160,223.891836,164.489328,197.450415,209.613063,182.737415,224.557110,151.095332,201.433348,147.735354,163.163628,167.412285,178.804095,161.251106,178.777444,155.879436,5.0


In [41]:
def predict_ev_charging_throttle(cadaster, hour, buffer=1.15, printing=False): #hour should be given in 0..23 range
  #Retrieve cadaster's known parameters, used for model predictions
  c_public_chargers = all_substations.loc[all_substations['cadaster'] == cadaster, 'public_chargers'].iloc[0]
  c_home_chargers = all_substations.loc[all_substations['cadaster'] == cadaster, 'home_chargers'].iloc[0]
  c_baseline = all_substations.loc[all_substations['cadaster'] == cadaster, str(hour)+'_y'].iloc[0]
  c_max_current = all_substations.loc[all_substations['cadaster'] == cadaster, 'max_current'].iloc[0]
  #Check that parameters were correctly retrieved
  #print(c_public_chargers, c_home_chargers, c_baseline)

  #Using our model to predict current usage for the hour
  hour_model = models[hour]
  pred = hour_model.predict([[c_public_chargers, c_home_chargers, c_baseline, c_max_current]])[0]
  #print(c_max_current, pred, all_substations.loc[all_substations['cadaster'] == cadaster, str(hour)+'_current'].iloc[0]) #max_current, predicted current, and actual current

  #If max current allows, we won't throttle. If throtling is necessary, we will do it equally for all chargers.
  if c_max_current > pred:
    if printing:
      print("Predicting hour",i,"charging does not need to be throttled.")
    return 0
  else:
    throttle_percent = (pred - c_max_current)*buffer/(c_public_chargers + c_home_chargers)
    if printing:
      print("Predicting hour",i,"charging needs to be throttled by", round(throttle_percent,3), "kW per charging station.")
    return round(throttle_percent,3)

In [32]:
def actual_throtling_need(cadaster, hour, printing=False): #hour should be given in 0..23 range
  c_total_chargers = all_substations.loc[all_substations['cadaster'] == cadaster, 'total_chargers'].iloc[0]
  c_actual_current = all_substations.loc[all_substations['cadaster'] == cadaster, str(hour)+'_current'].iloc[0]
  c_max_current = all_substations.loc[all_substations['cadaster'] == cadaster, 'max_current'].iloc[0]

  if c_max_current > c_actual_current:
    if printing:
      print("Hour",i,"charging does not need to be throttled.")
    return 0
  else:
    throttle_percent = (c_actual_current - c_max_current)/(c_total_chargers)
    if printing:
      print("Hour",i,"charging needs to be throttled by", round(throttle_percent,3), "kW per charging station.")
    return round(throttle_percent,3)

In [38]:
# Run only if you want to find new, better buffer
# print("Precicted throttle | Actual needed throttle")
# for i in range(24):
#   print(predict_ev_charging_throttle("79507:022:0008", i), "|", actual_throtling_need("79507:022:0008", i))
buffers = []
for buf in range(100,120,2):
  predicted_throttles = []
  actual_throttles = []
  for cadaster in all_substations["cadaster"]:  
    for i in range(24):
      predicted_throttles.append(predict_ev_charging_throttle(cadaster, i, buf/100))
      actual_throttles.append(actual_throtling_need(cadaster, i))
  buf_data = [buf/100, mean_squared_error(actual_throttles, predicted_throttles, squared=False)]
  print(buf_data)
  buffers.append(buf_data)

[1.0, 3.101179688172443]
[1.02, 3.0652549116547263]
[1.04, 3.034860049907165]
[1.06, 3.010092775836676]
[1.08, 2.991129904167254]
[1.1, 2.9781026430392687]
[1.12, 2.971039905950061]
[1.14, 2.970030291879981]
[1.16, 2.975052483638802]
[1.18, 2.986074404858551]


In [52]:
predicted_throttles = []
actual_throttles = []
throttles = pd.DataFrame(columns=["cadaster", "hour", "predicted_throttle", "actual_throttle"])
for cadaster in all_substations["cadaster"]:  
  for i in range(24):
    pred = predict_ev_charging_throttle(cadaster, i)
    actual = actual_throtling_need(cadaster, i)
    predicted_throttles.append(pred)
    actual_throttles.append(actual)
    throttles = throttles.append(pd.DataFrame([[cadaster, i, pred, actual]], columns=["cadaster", "hour", "predicted_throttle", "actual_throttle"]))
mean_squared_error(actual_throttles, predicted_throttles, squared=False)

2.9717841366685693

In [90]:
valepidi_x = ['0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x',
       '8_x', '9_x', '10_x', '11_x', '12_x', '13_x', '14_x', '15_x', '16_x',
       '17_x', '18_x', '19_x', '20_x', '21_x', '22_x', '23_x']
oigetpidi_x = ['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18',
       'x_19', 'x_20', 'x_21', 'x_22', 'x_23']

valepidi_y = ['0_y', '1_y',
       '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y',
       '12_y', '13_y', '14_y', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y',
       '21_y', '22_y', '23_y']
oigetpidi_y = ['y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'y_7', 'y_8', 'y_9',
       'y_10', 'y_11', 'y_12', 'y_13', 'y_14', 'y_15', 'y_16', 'y_17', 'y_18',
       'y_19', 'y_20', 'y_21', 'y_22', 'y_23']

vaheta_x = {valepidi_x[i]: oigetpidi_x[i] for i in range(len(valepidi_x))} 
vaheta_y = {valepidi_y[i]: oigetpidi_y[i] for i in range(len(valepidi_y))} 

In [99]:
all_substations.rename(columns=vaheta_x, inplace=True)
all_substations.rename(columns=vaheta_y, inplace=True)

In [128]:
results = pd.wide_to_long(all_substations, ["x_", "y_"], i="cadaster", j="hour")

results.rename(columns={"x_":"ev_charging", "y_":"baseload"}, inplace=True)
results=results.drop(columns=['0_current', '10_current', '11_current', '12_current', '13_current', '14_current', '15_current',
 '16_current', '17_current', '18_current', '19_current', '1_current', '20_current', '21_current', '22_current',
 '23_current', '2_current', '3_current', '4_current', '5_current', '6_current', '7_current', '8_current', '9_current',
 'Unnamed: 0', 'Unnamed: 0.1'])

In [134]:
results = pd.merge(results, throttles,  how='left', left_on=['cadaster','hour'], right_on = ['cadaster','hour'])

In [177]:
predicted_ev_charging = []
i = 1
for row in results.iterrows():
  if i == 1:
    print(row[1]) #0 cadaster, 1 hour, 7 max_current, 10 public_chargers, 12 home chargers, 14 baseload
    i+=1

  predicted_ev_charging.append(models[row[1][1]].predict([[row[1][10],row[1][12],row[1][14],row[1][7]]])[0])
  #predicted_ev_charging.append(model[i].predict(results[["public_chargers", "home_chargers", "baseload", "max_current"]]))

cadaster                                                    79301:001:0342
hour                                                                     0
index                                                                  138
x                                                              6.47475e+06
address                  Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...
total_chargers                                                           1
max_current                                                            100
latitude                                                           58.3833
y                                                                   661647
public_chargers                                                          0
longitude                                                          26.7641
home_chargers                                                            1
ev_charging                                                            7.2
baseload                 

In [168]:
results["predicted_ev_charging"] = predicted_ev_charging

In [169]:
results

,cadaster,hour,index,x,address,total_chargers,max_current,latitude,y,public_chargers,longitude,home_chargers,ev_charging,baseload,predicted_throttle,actual_throttle,predicted_ev_charging
0,79301:001:0342,0,138,6474748.69,"Juure, Jaamamõisa, Tartu linn, Tartu, Tartu li...",1.0,100,58.383253,661646.72,0.0,26.764144,1.0,7.2,43.085689,0,0,27.561710
1,79502:019:0018,0,15,6473103.15,"33, Viljandi mnt, Maarjamõisa, Tartu linn, Tar...",7.0,60,58.370127,657342.63,2.0,26.689523,5.0,84.2,79.995283,3.274,14.885,27.561710
2,79512:032:0012,0,193,6475460.14,"68a/3, Puiestee, Raadi-Kruusamäe, Tartu linn, ...",1.0,160,58.390462,659468.49,1.0,26.727509,0.0,0.0,36.859454,0,0,28.053823
3,79514:014:0021,0,168,6474849.44,"16a, Roosi, Ülejõe, Tartu linn, Tartu, Tartu l...",5.0,160,58.385042,659676.18,1.0,26.730641,4.0,51.7,25.561742,0,0,27.561710
4,79507:022:0008,0,59,6474067.16,"2, Juhan Liivi, Kesklinn, Tartu linn, Tartu, T...",5.0,160,58.378215,658809.66,3.0,26.714904,2.0,13.8,152.551035,0,1.27,27.561710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,79301:001:0749,23,194,6475398.26,"2a, Lubja, Ülejõe, Tartu linn, Tartu, Tartu li...",5.0,100,58.390196,658858.08,2.0,26.716788,3.0,0.0,136.501670,9.381,7.3,35.835917
884,79510:033:0012,23,103,6470566.42,"59B, Jalaka, Ropka tööstusrajoon, Tartu linn, ...",2.0,100,58.346660,659376.72,1.0,26.722654,1.0,0.0,106.508075,5.805,3.254,35.835917
885,79509:019:0001,23,186,6470545.14,"37, Voolu, Variku, Tartu linn, Tartu, Tartu li...",4.0,100,58.346618,658217.03,0.0,26.702650,4.0,22.0,67.630290,0,0,38.892419
886,79510:022:0016,23,80,6470989.73,"12A, Sepa, Ropka, Tartu linn, Tartu, Tartu lin...",2.0,100,58.350471,659324.71,0.0,26.722083,2.0,6.6,116.382758,8.006,11.491,38.892419


In [181]:
throttles.to_csv("throttle_data.csv", sep=";")
results.to_csv("rf_results.csv", sep=";")
train.to_csv("lr_train.csv", sep=";")
test.to_csv("lr_test.csv", sep=";")